<DIV ALIGN=CENTER>

# Data Pre-Processing- Fundamentals-II
## Dr. Utku Pamuksuz 
### pamuksuz@uchicago.edu

  
</DIV>  
-----
-----


## About

In this session, we will examine some high level data pre-processing. 
Pre-processing is one of the most time consuming tasks 
among other data mining and machine learning processes. We will analyze the airline data
that is frequently used in data science training curriculums. These data are located at
canvas, and we will specifically analyze the
`flights.csv` file. We will handle missing data, learn categorical value reactions and get familiar with DateTime index. Pandas is a data structure, important and valuable to process/mine data frames efficiently in Python.

In the end of this exercise, we will have a dataframe that contains a DateTime Index, no
missing values, and has the airport codes represented by categoricals.
We could save this data for latter analysis, or being applying learning
algorithms.


-----

In [1]:
# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd 


filename = '../../../Week-6/data/flights.csv'

# Read only selected columns for all rows.

usethis = (1, 2, 4, 14, 15, 16, 17, 18)
column_names = ['Month', 'Day', 'dTime', 'aDelay', 'dDelay', 'Depart', 'Arrive', 'Distance']

alldata = pd.read_csv(filename, header=0, na_values=['NA'], usecols=usethis, names=column_names)


FileNotFoundError: [Errno 2] File ../../../Week-6/data/flights.csv does not exist: '../../../Week-6/data/flights.csv'

-----

When we first read a file in Pandas dataframe(df), we will first explore
the data. `describe` method
provides a statistical summary of the data. In the example below, we have six
numerical columns that have minimum and maximum values, as
well as basic statistical summaries.

-----

In [ ]:
alldata.describe()

-----

### Missing Values

One issue, however, is the different number of rows for the different
columns. In fact, there are three different values: 5967780, 5736582,
and 5723673. We will not want to perform any analysis on a row
that does not have a valid departure time, we will first drop any row that does not
have a valid value in this column.

-----

In [ ]:
# Drop any row that is missing a departure time
#axis=0 is row, axis=1 is column

alldata.dropna(axis=0, subset=['dTime'], inplace=True)

-----

Now we can explore the number of rows for each column by employing the
`count` method. In this case, we see that the `aDelay` column is still
missing some values. In this case, we can fill these missing values with
another value. There are a number of options for accomplishing this
task, including replacing with a default value (for example, we could
use the departure delay), interpolation, or even imputation (where we
might determine a mean value for flights in this distance range or from
the departure airport). In our case, we can simply assume the
arrival delay is zero for any missing value.

-----

In [ ]:
# Now display number of 'ok' values for each column.
alldata.count()

In [ ]:
# Now replace missing values (which are all in Arrival Delay column)
# with 0, note we could use another value, such as the departure delay.
#values={'aDelay':0}

alldata.fillna(value=0, axis=0, inplace=True)

#values = {'aDelay': 0, 'Distance': 1, 'C': 2, 'D': 3}

-----

We now create a new df to hold our final data set. In this case,
we start by copying over the three columns that won't change.

-----

In [ ]:
# First create a new DataFrame
# For now, simply copy over the columns, but we could
# convert them to integers (for number, the number of minutes in the delay)
# and change data type to save memory.

newdata = alldata[['aDelay', 'dDelay', 'Distance']]

-----

### Categorical Values

 `describe` method didn't show `Depart`
and `Arrive` columns at the outset. Since they are interpreted as
string values, Pandas does not perform any statistical analysis of the
value. Since there are a limited number of airports that are contained
in our data, we can allow Pandas to map the Airport codes contained in
the `Depart` and `Arrive` columns to numerical representations by
employing [categorical][pdc] values. In this case, we can do this easily
by copying these columns over to our new DataFrame, and specifying that
they should be converted to type `categorical`.

To demonstrate the set of values that we will need to map to
categoricals, we can display all unique airport codes in the `Depart`
column.

-----
[pdc]: http://pandas.pydata.org/pandas-docs/stable/categorical.html

In [ ]:
alldata['Depart'].unique()
#alldata.dtypes
#type (alldata.Depart[0])

In [ ]:
# Now copy over the departure and Arrival columns, but change data type to categoricals. 

newdata['Depart'] = alldata['Depart'].astype('category')
newdata['Arrive'] = alldata['Arrive'].astype('category')

In [ ]:
alldata.head()

In [ ]:
newdata.head()

-----

### DateTime

Last but not least, how do we handle times?. Pandas has strong support for
[dates and times][pdt]. In this context, we can create a new
`DateTime` column. While it is generally easier to create a `DateTime`
column when reading a CSV file, it is not easy to account for missing
values during this process. In that case, one could read in the data,
clean missing values, and write the data back to a CSV (or other format)
file to enable the traditional data parsing to be performed.

In this case, however, we will instead create several new columns from
the original data set, and use these new columns to create a `DateTime`
column. First, we create a new `Year` column, which in this case we hard
code to `2001`, which is the year corresponding to these data. Second,
we copy over the `Month` and `Day` columns. Finally, we convert the
encoded departure time column to an `Hour` and `Min`. Before any new
processing, we first display the last few rows of our new DataFrame
before computing summary statistics for the new DataFrame to verify our
work.

-----

[pdt]: http://pandas.pydata.org/pandas-docs/stable/timeseries.html

In [ ]:
newdata['Year'] = 2001
newdata['Month'] = alldata.Month
newdata['Day'] = alldata.Day
newdata['Hour'] = (alldata.dTime/100).astype(int)
newdata['Min'] = (alldata.dTime - 100*(alldata.dTime/100.).astype(int)).astype(int)

In [ ]:
newdata.tail()

In [ ]:
newdata.describe()

----
### OPTIONAL
Now we can turn our attention to creating a `DateTime` column in our new
DataFrame. However, in the output of the `describe` method, you should
notice that the `Hour` column has a maximum of 24 hours. Our method of
creating a `DateTime` value employs the Python [`strftime`][pst] string
date/time representation. This representation does not allow values
higher than 23 for an hour, hence we have to modify our data. For this
demonstration, we will simply subtract one minute. In a production
system, you might instead convert the `DateTime` to the next day.

Once these columns have been modified appropriately, we create a
DateTime by using the `to_datetime` method. We first combine our date
and time DataFrame columns into a new string value, and specify the
appropriate format string to decode these values. After creating this
new column, we indicate that it should be used as the index for the new
DateFrame. Finally, we display the last few rows of the new DataFrame to
show the values.

-----
[pst]: http://strftime.org

In [3]:
newdata.loc[newdata.Hour == 24, 'Min'] = 59
newdata.loc[newdata.Hour == 24, 'Hour'] = 23


NameError: name 'newdata' is not defined

In [ ]:
newdata['DTI'] = pdt.o_datetime(newdata.Year * 100000000 + 
                                newdata.Month * 1000000 + 
                                newdata.Day * 10000 + 
                                newdata.Hour * 100 + 
                                newdata.Min, format="%Y%m%d%H%M")

In [ ]:
newdata.set_index('DTI', inplace=True)

In [ ]:
newdata.tail()